In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor


In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
X = data.drop(['PPV','Blast NO','B','S'],axis=1)
Y = data['PPV']

,Blast NO,B,S,ST,MC,PF,HD,BS,D,PPV
count,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.00000,135.000000,135.000000,135.000000
mean,68.000000,2.954519,3.623333,3.685185,915.002222,0.664889,12.77037,0.834000,576.758593,1.445844
std,39.115214,0.553385,1.090269,1.275107,931.259906,0.209366,4.97691,0.072813,410.371630,1.224736
min,1.000000,2.500000,3.000000,2.500000,21.016000,0.210000,4.75000,0.620000,23.000000,0.030000
25%,34.500000,2.660000,3.050000,3.000000,76.000000,0.410000,11.00000,0.830000,216.720000,0.445000
50%,68.000000,2.660000,3.050000,3.500000,567.500000,0.790000,12.50000,0.870000,460.000000,0.980000
75%,101.500000,3.670000,4.200000,3.500000,1693.500000,0.790000,12.50000,0.870000,1020.000000,2.230000
max,135.000000,4.000000,6.500000,7.000000,3000.000000,0.830000,25.00000,0.870000,1500.000000,4.760000


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

In [ ]:
DNN_model = Sequential()

# The Input Layer :
DNN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
DNN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
DNN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
DNN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
DNN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
DNN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
DNN_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               896       
                                                                 
 dense_6 (Dense)             (None, 256)               33024     
                                                                 
 dense_7 (Dense)             (None, 256)               65792     
                                                                 
 dense_8 (Dense)             (None, 256)               65792     
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 165,761
Trainable params: 165,761
Non-trainable params: 0
_________________________________________________________________


In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(X_train, Y_train, epochs=2000, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Streaming output truncated to the last 5000 lines.
Epoch 251/1500
1/3 [=========>....................] - ETA: 0s - loss: 0.3407 - mean_absolute_error: 0.3407
Epoch 251: val_loss did not improve from 0.45130
3/3 [==============================] - 0s 21ms/step - loss: 0.4237 - mean_absolute_error: 0.4237 - val_loss: 0.4662 - val_mean_absolute_error: 0.4662
Epoch 252/1500
1/3 [=========>....................] - ETA: 0s - loss: 0.5528 - mean_absolute_error: 0.5528
Epoch 252: val_loss did not improve from 0.45130
3/3 [==============================] - 0s 23ms/step - loss: 0.4471 - mean_absolute_error: 0.4471 - val_loss: 0.4866 - val_mean_absolute_error: 0.4866
Epoch 253/1500
1/3 [=========>....................] - ETA: 0s - loss: 0.3707 - mean_absolute_error: 0.3707
Epoch 253: val_loss did not improve from 0.45130
3/3 [==============================] - 0s 21ms/step - loss: 0.4078 - mean_absolute_error: 0.4078 - val_loss: 0.5044 - val_mean_absolute_error: 0.5044
Epoch 254/1500
1/3 [=========>.

In [ ]:
predictions = DNN_model.predict(X_test)

In [ ]:
Y_test

98     0.97
67     2.30
105    0.69
19     1.16
42     0.21
62     3.80
12     3.05
110    0.87
125    0.90
128    0.85
44     1.22
40     0.09
27     2.94
100    0.46
119    0.69
31     1.54
51     0.03
4      4.38
84     0.26
81     1.75
85     0.20
26     1.78
16     4.70
18     2.38
10     4.25
111    0.30
45     1.30
Name: PPV, dtype: float64

In [ ]:
predictions

array([[0.32790577],
       [3.235709  ],
       [0.83566016],
       [1.528728  ],
       [0.31580636],
       [4.1852117 ],
       [3.2470253 ],
       [0.37479672],
       [2.6480498 ],
       [0.8885817 ],
       [0.42366034],
       [0.16700797],
       [3.2982585 ],
       [0.3080701 ],
       [1.4939189 ],
       [1.0669839 ],
       [0.46060935],
       [2.1824937 ],
       [0.38665223],
       [1.7788775 ],
       [0.30125913],
       [2.1023293 ],
       [2.8288736 ],
       [1.8474894 ],
       [2.8937373 ],
       [0.77251595],
       [0.40388316]], dtype=float32)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(Y_test,predictions)

0.5947521005957215

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
np.sqrt(mean_squared_error(Y_test,predictions))

0.8255921899690004

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(Y_test,predictions)

0.6443405962704882

In [ ]:
testing = pd.read_csv('test.csv')

FileNotFoundError: ignored

In [ ]:
pred = NN_model.predict(testing)

In [ ]:
pred

In [ ]:
from math import tanh

def expression (ST, MC, PF, HD, BS, D) : 

    scaled_ST=(ST-3.68519)/1.27511
    scaled_MC=(MC-915.002)/931.26
    scaled_PF=(PF-0.664889)/0.209366
    scaled_HD=(HD-12.7704)/4.97691
    scaled_BS=(BS-0.834)/0.0728134
    scaled_D=(D-576.759)/410.372
    y_1_1=tanh(-0.749817
    +0.86322*scaled_ST
    -0.839783*scaled_MC
    -0.905823*scaled_PF
    -0.882568*scaled_HD
    -0.327209*scaled_BS
    +0.829346*scaled_D)
    y_1_2=tanh(-0.20282
    -0.13446*scaled_ST
    +0.892273*scaled_MC
    +0.674316*scaled_PF
    +0.0684204*scaled_HD
    +0.684143*scaled_BS
    +0.387024*scaled_D)
    y_1_3=tanh(-0.204651
    -0.481689*scaled_ST
    -0.991333*scaled_MC
    +0.0511475*scaled_PF
    +0.909546*scaled_HD
    -0.202637*scaled_BS
    -0.517822*scaled_D)
    y_2_1=tanh(0.171082
    -0.489746*y_1_1
    +0.367981*y_1_2
    +0.890503*y_1_3)
    y_2_2=tanh(-0.129028
    +0.780396*y_1_1
    -0.985657*y_1_2
    +0.881897*y_1_3)
    y_2_3=tanh(0.203064
    +0.572266*y_1_1
    +0.15332*y_1_2
    -0.715149*y_1_3)
    y_3_1=tanh(-0.555359
    -0.234009*y_2_1
    -0.991455*y_2_2
    -0.164185*y_2_3)
    y_3_2=tanh(-0.83551
    +0.319824*y_2_1
    +0.710144*y_2_2
    -0.8703*y_2_3)
    y_3_3=tanh(0.62207
    +0.324097*y_2_1
    +0.382935*y_2_2
    +0.605347*y_2_3)
    scaled_PPV=(0.0602417
    +0.371216*y_3_1
    -0.714478*y_3_2
    +0.378967*y_3_3)
    (PPV) = (4.53044+24.6225*scaled_PPV)
    
    return PPV 


In [ ]:
expression(2.5,69.47,0.27,8,0.62,100.49)